The product manager asked you to analyze the purchases made and answer the following questions:

1. How many users do we have? who made a purchase only once? 

2. How many orders per month, on average, are not delivered for various reasons (display details by reasons)?

3. For each product, determine on which day of the week the product is most often bought. 

4. How many purchases does each user make on average per week (by months)? Do not forget that there may not be an integer number of weeks within a month. For example, November 2021 has 4.28 weeks. And within the metrics, this must be taken into account.

5. Using pandas, conduct a cohort analysis of users. Between January and December, identify the cohort with the highest retention for the 3rd month. A description of the approach can be found here.

Order statuses:
- created 
- approved 
- invoiced 
- processing
- shipped 
- delivered
- unavailable 
- canceled 

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt


In [2]:
customers = pd.read_csv('./olist_customers_dataset.csv',encoding='Windows-1251')

In [3]:
customers.head(5)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [10]:
customers.dtypes # show types of columns

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [11]:
customers.shape # show number of rows and columns

(99441, 5)

In [12]:
customers.isna().sum() # number of empty fields

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [13]:
customers.customer_unique_id.nunique()
#Count number of distinct elements

96096

In [4]:
orders = pd.read_csv('./olist_orders_dataset.csv',encoding='Windows-1251', parse_dates=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])

In [5]:
orders.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [20]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [21]:
orders.shape

(99441, 8)

In [22]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [23]:
orders.nunique()

order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_approved_at                90733
order_delivered_carrier_date     81018
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64

The number of orders and customers (not-unique) is same - 99441. It means that the customer is everyone who made a purchase.
Let's check gow many unique clients we have.

In [6]:
items = pd.read_csv('./olist_order_items_dataset.csv',encoding='Windows-1251', parse_dates=['shipping_limit_date'])

In [7]:
items.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [28]:
items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [29]:
items.shape

(112650, 7)

#### How many users do we have who made a purchase only once? 

In [30]:
customers_and_orders = pd.merge(customers, orders, on = 'customer_id', how = 'inner') 
#merge customers and orders to one df

In [31]:
customers_and_orders.shape

(99441, 12)

In [32]:
customers_and_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15


In [33]:
one_purch = customers_and_orders.query('order_status == "delivered"')\
                                .groupby('customer_unique_id', as_index = False)\
                                .agg({"customer_city":"count"})\
                                .rename(columns = {"customer_city":"qty"})

In [34]:
          
print('{:,} - total number customers who made 1 purchase;'.format(len(one_purch.query('qty == 1'))))

90,557 - total number customers who made 1 purchase;


In [35]:
print('{:,} - total number of purchases;'.format(orders.shape[0])) 

99,441 - total number of purchases;



If the store took the goods for delivery, then the payment was made and the order was not canceled by either the store or the buyer.
It turns out that 90557 people made one purchase, and only 99441 users. Therefore, 91% only bought once.

#### 2. How many orders per month, on average, are not delivered for various reasons (display details for reasons)?

Let's check availabe statuses

In [36]:
orders.order_status.unique() #show status names used in table

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

Let's change the order of order statuses: first, a successful process (ending with delivery to the client), after an exception

In [37]:
orders["order_status"] = orders["order_status"].astype("category")

orders["order_status"].cat.set_categories(["created", "approved", "invoiced", "processing", "shipped", "delivered", "canceled", "unavailable"], inplace=True)

We will select orders where there is no status that they have been delivered to the client

In [38]:
not_delivered_orders = orders[orders['order_delivered_customer_date'].isna()]\
                      .query('order_status == ["canceled","unavailable"]')

In [39]:
print('{:,} - total number of undelivered orders;'.format(not_delivered_orders.shape[0]))

1,228 - total number of undelivered orders;


In [40]:
not_delivered_orders.head(4)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaT,NaT,2017-12-05
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaT,NaT,2018-08-14
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaT,NaT,2018-02-16
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaT,2018-02-22


Add a column with the purchase date. We will consider orders according to the month of the payment.

In [41]:
not_delivered_orders['purch_date'] = pd.to_datetime(not_delivered_orders['order_approved_at']).dt.strftime('%Y-%m')

In [42]:
not_delivered_orders.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purch_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaT,NaT,2017-12-05,2017-11
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaT,NaT,2018-08-14,2018-08
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaT,NaT,2018-02-16,2018-01
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaT,2018-02-22,2018-01
687,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaT,NaT,2017-09-05,2017-08


In [43]:
nd_orders = not_delivered_orders.pivot_table(index= "purch_date", columns = ["order_status"], aggfunc = {'order_id': "count"})\
.fillna(0)
# group (pivot) the table where rows: purchase dates, columns: status

In [44]:
nd_orders

order_id                                                          \
order_status  created approved invoiced processing shipped delivered canceled   
purch_date                                                                      
2016-10             0        0        0          0       0         0       15   
2017-01             0        0        0          0       0         0        2   
2017-02             0        0        0          0       0         0       15   
2017-03             0        0        0          0       0         0       31   
2017-04             0        0        0          0       0         0       13   
2017-05             0        0        0          0       0         0       19   
2017-06             0        0        0          0       0         0       14   
2017-07             0        0        0          0       0         0       23   
2017-08             0        0        0          0       0         0       21   
2017-09             0        0        0          0       0         0       16   
2017-10             0        0        0          0       0         0       21   
2017-11             0        0        0          0       0         0       29   
2017-12             0        0        0          0       0         0        7   
2018-01             0        0        0          0       0         0       32   
2018-02             0        0        0          0       0         0       70   
2018-03             0        0        0          0       0         0       23   
2018-04             0        0        0          0       0         0       14   
2018-05             0        0        0          0       0         0       24   
2018-06             0        0        0          0       0         0       18   
2018-07             0        0        0          0       0         0       39   
2018-08             0        0        0          0       0         0       32   

                          
order_status unavailable  
purch_date                
2016-10                6  
2017-01                8  
2017-02               47  
2017-03               32  
2017-04                9  
2017-05               32  
2017-06               24  
2017-07               52  
2017-08               31  
2017-09               37  
2017-10               57  
2017-11               86  
2017-12               43  
2018-01               47  
2018-02               31  
2018-03               16  
2018-04                6  
2018-05               16  
2018-06                4  
2018-07               12  
2018-08               13

As we have to get the average number of orders that were not delivered, we have to get the average result of 2 columns (canceled, unavailable)

In [45]:
nd_orders.columns = ['created', 'approved','invoiced', 'processing','shipped', 'delivered','canceled', 'unavailable']

In [46]:
nd_orders = nd_orders[['canceled', 'unavailable']].copy() #select 2 columns with required statuses

In [47]:
nd_orders['avg_ttl_nd_orders']= ((nd_orders['canceled'] + nd_orders['unavailable'])/2).round()  # calculate the average

In [48]:
nd_orders

,canceled,unavailable,avg_ttl_nd_orders
purch_date,,,
2016-10,15,6,10.0
2017-01,2,8,5.0
2017-02,15,47,31.0
2017-03,31,32,32.0
2017-04,13,9,11.0
2017-05,19,32,26.0
2017-06,14,24,19.0
2017-07,23,52,38.0
2017-08,21,31,26.0


We consider that orders cannot be delivered for 2 main reasons: too long to wait - and the buyer cancels the order. Another case happens and if the seller himself cancels the purchase. 
And the purpose of the request is to understand the situation on orders that were not delivered at all.
 
We consider that the status characterizing the cancellation is unavailable, canceled.

#### 3. For each product, determine on which day of the week the product is most often bought

In [49]:
orders_and_items = pd.merge(orders,items,on = 'order_id', how = 'left') #merge orders and items

In [50]:
orders_and_items.shape

(113425, 14)

In [51]:
orders_and_items['Purch_Weekday']= pd.to_datetime(orders_and_items['order_purchase_timestamp']).dt.day_name()
# add the new column with the name od the day

In [52]:
orders_and_items.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,Purch_Weekday
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,Monday
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,Tuesday
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,Wednesday
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,Saturday
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,Tuesday


In [53]:
products_by_weekday = orders_and_items.groupby(['product_id','Purch_Weekday'], as_index = False)\
                                      .agg({"order_id":"count"})\
                                      .sort_values(by =['order_id'], ascending=True)\
                                      .rename(columns = {'order_id':'orders'})\
                                      .pivot_table(index= "product_id", columns = ["Purch_Weekday"], aggfunc = {'orders': "count"})\
                                      .fillna(0)


In [54]:
products_by_weekday

orders                                  \
Purch_Weekday                    Friday Monday Saturday Sunday Thursday   
product_id                                                                
00066f42aeeb9f3007548bb9d3f33c38    0.0    0.0      0.0    1.0      0.0   
00088930e925c41fd95ebfe695fd2655    0.0    0.0      0.0    0.0      0.0   
0009406fd7479715e4bef61dd91f2462    0.0    0.0      0.0    0.0      1.0   
000b8f95fcb9e0096488278317764d19    1.0    0.0      0.0    0.0      0.0   
000d9be29b5207b54e86aa1b1ac54872    0.0    0.0      0.0    0.0      0.0   
...                                 ...    ...      ...    ...      ...   
fff6177642830a9a94a0f2cba5e476d1    0.0    0.0      1.0    1.0      0.0   
fff81cc3158d2725c0655ab9ba0f712c    0.0    1.0      0.0    0.0      0.0   
fff9553ac224cec9d15d49f5a263411f    1.0    0.0      0.0    0.0      0.0   
fffdb2d0ec8d6a61f0a0a0db3f25b441    1.0    0.0      0.0    1.0      1.0   
fffe9eeff12fcbd74a2f2b007dde0c58    0.0    0.0      0.0    0.0      0.0   

                                                    
Purch_Weekday                    Tuesday Wednesday  
product_id                                          
00066f42aeeb9f3007548bb9d3f33c38     0.0       0.0  
00088930e925c41fd95ebfe695fd2655     1.0       0.0  
0009406fd7479715e4bef61dd91f2462     0.0       0.0  
000b8f95fcb9e0096488278317764d19     0.0       1.0  
000d9be29b5207b54e86aa1b1ac54872     1.0       0.0  
...                                  ...       ...  
fff6177642830a9a94a0f2cba5e476d1     0.0       0.0  
fff81cc3158d2725c0655ab9ba0f712c     0.0       0.0  
fff9553ac224cec9d15d49f5a263411f     0.0       0.0  
fffdb2d0ec8d6a61f0a0a0db3f25b441     1.0       0.0  
fffe9eeff12fcbd74a2f2b007dde0c58     0.0       1.0  

[32951 rows x 7 columns]

Or another way to get exact day for each product

In [55]:
orders_and_items

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,Purch_Weekday
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,Monday
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,Tuesday
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,Wednesday
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,Saturday
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113420,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,1.0,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10,Tuesday
113421,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,1.0,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02,Sunday
113422,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,1.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59,Monday
113423,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,2.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59,Monday


To determine on which day of the week a certain product (product_id) was most often bought, you need to:
1. Group the dataframe by product and day of the week and count the number of records. Thus, we obtain for each product the number of purchases by day of the week;
2. Sort the resulting table in descending order of the number of purchases;
3. Leave only the first entries for each item - this will correspond to the day of the week with the maximum number of purchases for each item.

In [56]:
exact_day = orders_and_items.groupby(['product_id','Purch_Weekday'], as_index = False)\
                            .agg({"order_status":"count"})\
                            .sort_values(by='order_status', ascending = False)\
                            .rename(columns = {'order_status':'orders'})\
                            .groupby('product_id')\
                            .head(1)

In [57]:
exact_day

,product_id,Purch_Weekday,orders
15902,422879e10f46682990de24d770e7f83d,Wednesday,93
36513,99a4788cb24856965c36a24e339b6058,Monday,92
41045,aca2eb7d00ea1a7b8ebd4e68314663af,Thursday,89
20127,53b36df67ebb7c41585e8d54d6772e08,Tuesday,76
13537,389d119b48cf3043d311335e499d9c6b,Thursday,67
...,...,...,...
24030,6432d844f2be41bb6f2fd279e23415da,Friday,1
24009,6421ae053bb80dff3ceae1f217f54ed4,Monday,1
24010,642247ccf62585635cefa1bf09f59b94,Sunday,1
24022,642e125cbc752b4eeb8faf442319c39f,Thursday,1


#### 4. How many purchases does each user make on average per week (by months)? 
Do not forget that there may not be an integer number of weeks within a month. For example, November 2021 has 4.28 weeks. And within the metrics, this must be taken into account.

In [58]:
all_df = pd.merge(customers_and_orders,items,on = 'order_id', how = 'left')

In [59]:
#We will consider order_approved_at as the time of purchase

In [60]:
all_df['weeks_in_month'] = ((all_df['order_approved_at'].dt.days_in_month)/7).round(1) #calculate the number of weeks in month

In [61]:
all_df.head(5)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,weeks_in_month
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,4.4
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48,4.4
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79,4.4
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36,4.4
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,1.0,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25,4.4


In [62]:
all_df['purchase_month'] = pd.to_datetime(all_df['order_approved_at'].dt.strftime('%Y-%m'))

In [63]:
orders_by_customers = all_df.groupby(['purchase_month','customer_unique_id', 'weeks_in_month'], as_index=False)\
                            .agg({'product_id':'count'})\
                            .rename(columns={'product_id':'orders_qty'})

In [64]:
orders_by_customers['avg_orders']=(orders_by_customers['orders_qty']/orders_by_customers['weeks_in_month']).round(2)

In [65]:
orders_by_customers.sort_values(['purchase_month','customer_unique_id'], ascending= True).head(2)

,purchase_month,customer_unique_id,weeks_in_month,orders_qty,avg_orders
0,2016-09-01,830d5b7aaa3b6f1e9ad63703bec97d23,4.3,3,0.70
1,2016-10-01,0032c76b20340da25249092a268ce66c,4.4,1,0.23



#### 5. Identify the cohort with the highest retention for the 3rd month in the period between January and December.


- Retention N = Number of users logged in on N day / Number of users who installed the app N days ago * 100%.
___
- Sign of cohort formation - first purchase (order_purchase_timestamp);
- Cohort size - 1 month Reporting period - from January 2017 to December 2017

In [66]:
customers_and_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15


In [67]:
customers_and_orders.order_purchase_timestamp.min()

Timestamp('2016-09-04 21:15:19')

In [68]:
customers_and_orders.order_purchase_timestamp.max()

Timestamp('2018-10-17 17:30:18')

Let's select only the required columns

In [69]:
new = customers_and_orders[['customer_unique_id', 'order_id', 'order_purchase_timestamp', 'order_approved_at']].copy()

In [70]:
new.head(6)

,customer_unique_id,order_id,order_purchase_timestamp,order_approved_at
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,2017-05-16 15:22:12
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,2018-01-12 20:58:32
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,2018-05-20 16:19:10
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,2018-03-13 17:29:19
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,2018-07-29 10:10:09
5,4c93744516667ad3b8f1fb645a3116a4,5741ea1f91b5fbab2bd2dc653a5b5099,2017-09-14 18:14:31,2017-09-14 18:25:11


In [71]:
# We need to distribute our customers according to cohorts by the date of purchase

In [72]:
new['order_month'] = new['order_purchase_timestamp'].dt.to_period('M') # changes the date format

In [73]:
new = new.query('order_purchase_timestamp > "2016-12-31" and order_purchase_timestamp < "2018-01-01"')

For each unique user, determine the month of his first purchase and add this information to the 'first_order_month' column

In [74]:
first_order = new.groupby('customer_unique_id')\
                             .agg({"order_purchase_timestamp":"min"})\
                             .rename(columns={'order_purchase_timestamp':'first_order_month'})

In [75]:
first_order.first_order_month = first_order.first_order_month.dt.to_period('M')

In [76]:
purchases = new.merge(first_order, how='left', on='customer_unique_id')
purchases.head(5)

,customer_unique_id,order_id,order_purchase_timestamp,order_approved_at,order_month,first_order_month
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05,2017-05
1,4c93744516667ad3b8f1fb645a3116a4,5741ea1f91b5fbab2bd2dc653a5b5099,2017-09-14 18:14:31,2017-09-14 18:25:11,2017-09,2017-09
2,57b2a98a409812fe9618067b6b8ebe4f,1093c8304c7a003280dd34598194913d,2017-11-16 19:29:02,2017-11-16 19:55:41,2017-11,2017-11
3,2a7745e1ed516b289ed9b29c7d0539a5,8428e578bb1cf839ae26a6b7615502b9,2017-11-27 17:23:20,2017-11-27 18:01:20,2017-11,2017-11
4,918dc87cd72cd9f6ed4bd442ed785235,83d8f70418eefd96f37b483dff430305,2017-09-09 09:54:57,2017-09-09 10:10:17,2017-09,2017-09


In [77]:
# Let's determine the number of unique customers for the corresponding period
cohorts = purchases.groupby(['first_order_month', 'order_month'], as_index = False)\
                   .agg({'customer_unique_id': 'nunique'})
cohorts.rename(columns={'customer_unique_id': 'total_clients'}, inplace=True)

In [78]:
purchases.head(5)

,customer_unique_id,order_id,order_purchase_timestamp,order_approved_at,order_month,first_order_month
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05,2017-05
1,4c93744516667ad3b8f1fb645a3116a4,5741ea1f91b5fbab2bd2dc653a5b5099,2017-09-14 18:14:31,2017-09-14 18:25:11,2017-09,2017-09
2,57b2a98a409812fe9618067b6b8ebe4f,1093c8304c7a003280dd34598194913d,2017-11-16 19:29:02,2017-11-16 19:55:41,2017-11,2017-11
3,2a7745e1ed516b289ed9b29c7d0539a5,8428e578bb1cf839ae26a6b7615502b9,2017-11-27 17:23:20,2017-11-27 18:01:20,2017-11,2017-11
4,918dc87cd72cd9f6ed4bd442ed785235,83d8f70418eefd96f37b483dff430305,2017-09-09 09:54:57,2017-09-09 10:10:17,2017-09,2017-09


In [79]:
cohorts

,first_order_month,order_month,total_clients
0,2017-01,2017-01,765
1,2017-01,2017-02,3
2,2017-01,2017-03,2
3,2017-01,2017-04,1
4,2017-01,2017-05,3
...,...,...,...
72,2017-10,2017-11,31
73,2017-10,2017-12,11
74,2017-11,2017-11,7305
75,2017-11,2017-12,40


Let's find the number of users in each cohort

In [83]:
group_size = purchases.groupby('first_order_month', as_index=False)\
                      .agg({"order_id":"count"})\
                      .rename(columns ={'order_id':'ttl_users'})

In [84]:
group_size

,first_order_month,ttl_users
0,2017-01,820
1,2017-02,1813
2,2017-03,2748
3,2017-04,2447
4,2017-05,3772
5,2017-06,3272
6,2017-07,4037
7,2017-08,4338
8,2017-09,4269
9,2017-10,4579


In [85]:
retention_df = group_size.merge(cohorts, on='first_order_month')

In [86]:
retention_df

,first_order_month,ttl_users,order_month,total_clients
0,2017-01,820,2017-01,765
1,2017-01,820,2017-02,3
2,2017-01,820,2017-03,2
3,2017-01,820,2017-04,1
4,2017-01,820,2017-05,3
...,...,...,...,...
72,2017-10,4579,2017-11,31
73,2017-10,4579,2017-12,11
74,2017-11,7455,2017-11,7305
75,2017-11,7455,2017-12,40


In [87]:
retention_df['retention']=(retention_df['total_clients']/retention_df['ttl_users']).round(4)

In [88]:
retention_df = retention_df.rename(columns={'first_order_month':'cohort'})

In [89]:
cohort_pivot = retention_df.pivot_table(index = 'cohort',
                                     columns = 'order_month',
                                     values = 'retention')
cohort_pivot

order_month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
cohort,,,,,,,,,,,,
2017-01,0.9329,0.0037,0.0024,0.0012,0.0037,0.0012,0.0049,0.0012,0.0012,NaN,0.0037,0.0012
2017-02,NaN,0.9664,0.0022,0.0028,0.0011,0.0039,0.0011,0.0022,0.0017,0.0017,0.0022,0.0011
2017-03,NaN,NaN,0.9592,0.0047,0.0036,0.0036,0.0033,0.0015,0.0015,0.0029,0.0033,0.0007
2017-04,NaN,NaN,NaN,0.9616,0.0057,0.0020,0.0016,0.0033,0.0025,0.0033,0.0029,0.0029
2017-05,NaN,NaN,NaN,NaN,0.9533,0.0048,0.0048,0.0037,0.0029,0.0032,0.0040,0.0016
2017-06,NaN,NaN,NaN,NaN,NaN,0.9594,0.0046,0.0034,0.0040,0.0024,0.0037,0.0037
2017-07,NaN,NaN,NaN,NaN,NaN,NaN,0.9648,0.0050,0.0035,0.0025,0.0027,0.0020
2017-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9645,0.0067,0.0032,0.0025,0.0035
2017-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9677,0.0066,0.0052,0.0028


91% of users in tha dataset only bought once.
February'18 was the month with the highest value of cancelled orders, average orders that were not delivered that month was 50%.
As there is data for the full year only related to 2017 year, cohort-analysis was made for 2017 year.
2017-09 cohort has the highest retention.